# Prática - Mapeamento de Texturas + MVP

### Primeiro, vamos importar as bibliotecas necessárias.

In [ ]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from numpy import random
from PIL import Image

from shader_s import Shader
from objetos.objeto import Objeto
from file_loader import Loader

from objetos.grass.grass import Grass
from objetos.forest.forest_generator import ForestGenerator

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

altura = 700
largura = 700

window = glfw.create_window(largura, altura, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)


### Constroi e compila os shaders. Também "linka" eles ao programa

#### Novidade aqui: modularização dessa parte do código --- temos agora uma classe e arquivos próprios para os shaders (vs e fs)
Créditos: https://learnopengl.com

In [3]:
ourShader = Shader("vertex_shader.vs", "fragment_shader.fs")
ourShader.use()

program = ourShader.getProgram()

### Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

O file_loader carrega modelos a partir de arquivos no formato WaveFront (.obj).

Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file

In [4]:
loader = Loader()

In [ ]:
#gera os modelos 

grass = Grass(loader, 30, 30)
forest_gen = ForestGenerator(loader)
forest_gen.generate()


Processando modelo objetos/caixa/caixa.obj. Vertice inicial: 0
Processando modelo objetos/caixa/caixa.obj. Vertice final: 36
Processando modelo objetos/grass/grass.obj. Vertice inicial: 36
Processando modelo objetos/grass/grass.obj. Vertice final: 149796
Processando modelo objetos/forest/tree.obj. Vertice inicial: 149796
Processando modelo objetos/forest/tree.obj. Vertice final: 152610
Gerou 76 modelos do tipo objetos/forest/tree.obj.
Processando modelo objetos/forest/stone.obj. Vertice inicial: 152610
Processando modelo objetos/forest/stone.obj. Vertice final: 153306
Gerou 32 modelos do tipo objetos/forest/stone.obj.
Processando modelo objetos/forest/tree.obj. Vertice inicial: 153306
Processando modelo objetos/forest/tree.obj. Vertice final: 156120
Gerou 79 modelos do tipo objetos/forest/tree.obj.
Processando modelo objetos/forest/stone.obj. Vertice inicial: 156120
Processando modelo objetos/forest/stone.obj. Vertice final: 156816
Gerou 31 modelos do tipo objetos/forest/stone.obj.


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar dois slots (buffers): um para os vértices e outro para as texturas.

In [6]:
buffer_VBO = glGenBuffers(2)

### Enviando coordenadas de vértices para a GPU

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [7]:
vertices = np.zeros(len(loader.vertices_list), [("position", np.float32, 3)])
vertices['position'] = loader.vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

### Enviando coordenadas de textura para a GPU

In [8]:
textures = np.zeros(len(loader.textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = loader.textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")

glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [ ]:
# camera
cameraInitPos = glm.vec3(0.0, 8, 0.0)
cameraPos   = glm.vec3(cameraInitPos)
cameraFront = glm.vec3(0.0, 0.0, -1.0)
cameraUp    = glm.vec3(0.0, 1.0, 0.0)

firstMouse = True
yaw   = -90.0	# yaw is initialized to -90.0 degrees since a yaw of 0.0 results in a direction vector pointing to the right so we initially rotate a bit to the left.
pitch =  0.0
lastX =  largura / 2.0
lastY =  altura / 2.0
fov   =  45.0

# timing
deltaTime = 0.0	# time between current frame and last frame
lastFrame = 0.0


firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2





oscilation_velocity = 15
oscilation_amplitude = 0.4
player_speed = 50

camera_y_offset = 0
camera_run_timer = 0
def move_player(window):
    global cameraPos, cameraFront, cameraUp, camera_y_offset, camera_run_timer

    cameraSpeed = player_speed * deltaTime

    move_direction = glm.vec3(0)
    
    # Movimento para frente/trás (W/S) - restrito ao plano XZ
    if glfw.get_key(window, glfw.KEY_W) == glfw.PRESS:
        move_direction += glm.normalize(glm.vec3(cameraFront.x, 0.0, cameraFront.z))

    if glfw.get_key(window, glfw.KEY_S) == glfw.PRESS:
        move_direction += -glm.normalize(glm.vec3(cameraFront.x, 0.0, cameraFront.z))

    # Movimento lateral (A/D) - restrito ao plano XZ
    if glfw.get_key(window, glfw.KEY_A) == glfw.PRESS:
        right = glm.normalize(glm.cross(cameraFront, cameraUp))
        move_direction += -glm.vec3(right.x, 0.0, right.z)

    if glfw.get_key(window, glfw.KEY_D) == glfw.PRESS:
        right = glm.normalize(glm.cross(cameraFront, cameraUp))
        move_direction += glm.vec3(right.x, 0.0, right.z)
    
    cameraPos += cameraSpeed * move_direction

    # Aplica oscilação caso esteja se movendo
    # Oscilação da câmera ao correr
    if glm.length(move_direction) > 0.001:
        camera_run_timer += deltaTime * oscilation_velocity  # Velocidade da oscilação
        camera_y_offset = math.sin(camera_run_timer) * oscilation_amplitude  # Amplitude da oscilação
    else:
        # Reseta suavemente a oscilação quando parar
        camera_run_timer = 0
        camera_y_offset = glm.mix(camera_y_offset, 0.0, deltaTime * 10)

    # Aplica offset ao Y da posição da câmera
    cameraPos.y = cameraInitPos.y + camera_y_offset
        


        

def framebuffer_size_callback(window, largura, altura):

    # make sure the viewport matches the new window dimensions note that width and 
    # height will be significantly larger than specified on retina displays.
    glViewport(0, 0, largura, altura)

# glfw: whenever the mouse moves, this callback is called
# -------------------------------------------------------
def mouse_callback(window, xpos, ypos):
    global cameraFront, lastX, lastY, firstMouse, yaw, pitch
   
    if (firstMouse):

        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos # reversed since y-coordinates go from bottom to top
    lastX = xpos
    lastY = ypos

    sensitivity = 0.1 # change this value to your liking
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset
    pitch += yoffset

    # make sure that when pitch is out of bounds, screen doesn't get flipped
    if (pitch > 89.0):
        pitch = 89.0
    if (pitch < -89.0):
        pitch = -89.0

    front = glm.vec3()
    front.x = glm.cos(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    front.y = glm.sin(glm.radians(pitch))
    front.z = glm.sin(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

# glfw: whenever the mouse scroll wheel scrolls, this callback is called
# ----------------------------------------------------------------------
def scroll_callback(window, xoffset, yoffset):
    global fov

    fov -= yoffset
    if (fov < 1.0):
        fov = 1.0
    if (fov > 45.0):
        fov = 45.0
    
glfw.set_framebuffer_size_callback(window, framebuffer_size_callback)
glfw.set_cursor_pos_callback(window, mouse_callback)
glfw.set_scroll_callback(window, scroll_callback)

# tell GLFW to capture our mouse
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

### Nesse momento, nós exibimos a janela!


In [10]:
glfw.show_window(window)

### Funções Auxiliares

In [ ]:
#aplica o view e projection

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(fov), largura/altura, 0.1, 100.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

def set_view():
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

def set_projection():
    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)


#sai do tela com ESQ
def get_exit_event(window):
    if glfw.get_key(window, glfw.KEY_ESCAPE) == glfw.PRESS:
        glfw.set_window_should_close(window, True)

#conta o tempo independentemente do frame rate
#atualiza o deltaTime
def increase_time():
    global deltaTime, lastFrame
    currentFrame = glfw.get_time()
    deltaTime = currentFrame - lastFrame
    lastFrame = currentFrame

#modifica o polygonal mode com P
polygonal_mode = False 
def set_polygonal_mode(window):
    if glfw.get_key(window, glfw.KEY_P) == glfw.PRESS:
        polygonal_mode = not polygonal_mode
        if polygonal_mode:
            glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
        else:
            glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)



#renderiza os elementos da cena
def draw_scene(program):
    grass.draw(program)
    forest_gen.draw(program)



### Loop principal da janela.

In [ ]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

while not glfw.window_should_close(window): 
    increase_time() #conta o tempo

    get_exit_event(window) #sai do window com ESQ
    move_player(window) #aplica movimento na camera
    set_polygonal_mode(window) #define modo poligonal
    glfw.poll_events() #movimentos da camera (mouse)
       
    #inicializa a cena
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    

    draw_scene(program) #rederiza
    
    #aplica view e projection
    set_view()
    set_projection()
    
    
    glfw.swap_buffers(window)

glfw.terminate()